In [1]:
import logging
import sacrebleu
from pathlib import Path
import pandas as pd
import sys
from simpletransformers.t5 import T5Model, T5Args

In [2]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [3]:
model_args = T5Args()
model_args.max_length = 50
model_args.length_penalty = 2.0
model_args.repetition_penalty = 2.0
model_args.num_beams = 5
model_args.early_stopping = True
#model_args.do_sample = True
#model_args.top_p = 0.3
model_args.num_return_sequences = 5

In [4]:
#model = T5Model("mt5", "persiannlp/mt5-base-parsinlu-opus-translation_fa_en", args=model_args)
model_name = "outputs_t5_small_full_2020/"
task = "xWant"
model = T5Model("t5", model_name, args=model_args)

In [5]:
print("predicting")
#print(model.predict(["xReact: Ali buys a book",
#                     "xReact: Ali fell on his knees"]))

predicting


In [6]:
df = pd.read_csv("data/eval.tsv", sep="\t").astype(str)
# Prepare the data for testing
#df = df.groupby('input_text')['target_text'].apply(list)
def my_eval(df, prefix):
    df = df.groupby(['prefix','input_text'], as_index=False).agg({'target_text':lambda x: list(x)})
    truth_values = df.loc[df["prefix"] == prefix]["target_text"].tolist()
    input_values = df.loc[df["prefix"] == prefix]["input_text"].tolist()
    input_values = [prefix + ": " + str(input_text) for input_text in input_values]

    pred_values = model.predict(input_values)
    preds = [item[0] for item in pred_values]
    
    refs = [[] for i in range(5)]
    for item in truth_values:
        for i in range(5):
            if i < len(item):
                refs[i].append(item[i])
            else:
                refs[i].append('')  
    bleu_score = sacrebleu.corpus_bleu(preds, refs)
    return bleu_score, input_values, truth_values, pred_values
    
tasks = list(set(df["prefix"].tolist()))

In [7]:
bleu_score, input_values, truth_values, pred_values = my_eval(df, task)

print("BLEU for " + task + ":" + str(bleu_score.score))

sel_args = ["length_penalty", "repetition_penalty", "num_beams", "do_sample", "top_p", "top_k"]
Path(f'{model_name}/results/').mkdir(exist_ok=True, parents=True)
cc = 1
args = model_args.get_args_for_saving()
res_name = "res_" + task
for key, val in args.items():
    if key in sel_args:
        res_name += "_" + key + "_" + str(val)

with open(f'{model_name}/results/{res_name}.txt', 'w') as f:
        print("BLEU:" + str(bleu_score.score), file=f)
        for inp, truth, pred in zip(input_values, truth_values, pred_values):
            inp = inp.replace("as a result PersonX want","")
            cc += 1
            if cc < 10:
                print(inp,":", truth,"--", pred[0])
            print(inp,":", truth,"--", pred, file =f)
        print("Arguments:", file=f)
        for key, val in args.items():
            print(key, "=", val, file=f)

Generating outputs:   0%|          | 0/274 [00:00<?, ?it/s]

/home/pouramini/miniconda3/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:3221: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


Decoding outputs:   0%|          | 0/10960 [00:00<?, ?it/s]

BLEU for xWant:7.83849507565537
xWant: PersonX 'd better go : ['to escape from him', 'to resign his job', 'to leave on time', 'to arrive home', 'to relax and unwind', 'to walk away', 'not speak to anyone'] -- to take a shower
xWant: PersonX accepts PersonX's diploma : ['to attend  college', 'to get a job', 'get a job', 'apply for college', 'celebrate', 'breathe a sigh of relief'] -- to get a diploma
xWant: PersonX accepts PersonY invitation : ['go to engagement', 'make a note', 'to invite other people', 'to talk about the party', 'to go out with PersonY.', 'to drive to the event.'] -- to have a good time
xWant: PersonX accepts PersonY offer : ['to start the new relationship', 'to sign documents confirming the offer', 'to discuss with personY', 'to'] -- to thank PersonY for the offer
xWant: PersonX accepts PersonY's invitation : ['to honour person Y as a host', 'to attend a good seminar', 'bring gift', 'show up in the event'] -- to have a good time
xWant: PersonX accepts PersonY's offer